# Fase PreDB

El objetivo es preparar las tablas para nuestra Base de Datos
Vamos a tener las siguientes tablas y en el siguiente orden:
- fechas
- combustibles
- categorías
- unidades
- historico_precios

In [ ]:
# Librerías para el tratamiento y manipulación de datos
# -----------------------------------------------------------------------
import pandas as pd             # Manipulación de datos en estructuras DataFrame para análisis y transformación
import numpy as np              # Realización de cálculos numéricos y manipulación de arrays multidimensionales
import datetime                 # Manejo y procesamiento de fechas y horas, útil para organizar datos temporalmente

# Librerías del sistema y utilidades
# -----------------------------------------------------------------------
import os                       # Interactúa con el sistema operativo para gestionar rutas y archivos
import sys                      # Modifica las rutas de búsqueda de módulos y permite gestionar la ejecución del programa
sys.path.append("../")          # Añade el directorio padre al PATH para importar módulos personalizados

# Importación de funciones personalizadas para preparar datos antes de la carga en la base de datos
from src.SupportPreDB import (
    crear_dictio,               # Crea un diccionario para mapear valores en un DataFrame
    df_a_db                     # Mapea valores en un DataFrame usando un diccionario, preparando los datos para la base de datos
)

In [30]:
df_gas = pd.read_csv("../datos/01_PreciosGas/03_TablaPreciosGasFinal.csv",index_col="Unnamed: 0")
df_luz = pd.read_csv("../datos/02_PreciosLuz/02_PreciosLuzFinal.csv",index_col="Unnamed: 0")
df_gasolina = pd.read_csv("../datos/03_PreciosCombustible/01_TablaPreciosCombustible.csv",index_col="Unnamed: 0")
df_agua = pd.read_csv("../datos/04_PreciosAgua/01_PreciosAguaFinal.csv",index_col="Unnamed: 0")

# Tabla Fechas
Para esta tabla cogemos los valores únicos de las fechas de cualquiera de nuestras tablas

In [31]:
df_fechas = pd.DataFrame(df_gas["fecha"].unique())
df_fechas.index = df_fechas.index + 1
df_fechas.reset_index(inplace=True)
df_fechas = df_fechas.rename(columns={0:"fecha"})
df_fechas.sample()

,index,fecha
21,22,2020-10-01


In [32]:
df_fechas.to_csv("../datos/05_TablasPreDB/01_tabla_fechas.csv")

# Tabla combustibles
Para esta tabla sacamos los valores únicos del tipo combustible de la tabla "Combustibles"

In [33]:
df_combustibles = pd.DataFrame(df_gasolina["combustible"].unique())
df_combustibles = df_combustibles.rename(columns={0:"combustible"})
df_combustibles = pd.concat([pd.DataFrame({"index": [0], "combustible": ["N/A"]}), df_combustibles], ignore_index=True)
df_combustibles = df_combustibles.drop(columns="index")
df_combustibles.index = df_combustibles.index + 1
df_combustibles.reset_index(inplace=True)
df_combustibles

,index,combustible
0,1,N/A
1,2,Gasolina 95
2,3,Gasolina 98
3,4,Diésel
4,5,Diésel Premium


In [34]:
df_combustibles.to_csv("../datos/05_TablasPreDB/02_tabla_combustibles.csv")

# Tabla categorías
Categorías de los valores que estamos comprobando:
- luz
- gas
- combustible
- agua

In [35]:
categorias = ["luz","gas","combustible","agua"]
df_categorias = pd.DataFrame(categorias)
df_categorias.index = df_categorias.index + 1
df_categorias.reset_index(inplace=True)
df_categorias = df_categorias.rename(columns={0:"categoria"})
df_categorias

,index,categoria
0,1,luz
1,2,gas
2,3,combustible
3,4,agua


In [36]:
df_categorias.to_csv("../datos/05_TablasPreDB/03_tabla_categorias.csv")

# Tabla unidades
cada Precio tiene su unidad que son:
- €/L para la gasolina
- €/m3 para el agua
- €/MWh para la luz y el Gas

In [37]:
unidades = ["€/L","€/m3","€/MWh"]
df_unidades = pd.DataFrame(unidades)
df_unidades.index = df_unidades.index + 1
df_unidades.reset_index(inplace=True)
df_unidades = df_unidades.rename(columns={0:"unidad"})
df_unidades

,index,unidad
0,1,€/L
1,2,€/m3
2,3,€/MWh


In [38]:
df_unidades.to_csv("../datos/05_TablasPreDB/04_tabla_unidades.csv")

# Tabla Histórico_precios
Para esta tabla haremos un concat de todas las tablas que tenemos, primero las prepararemos para unirlas y después reemplazaremos los valores por ids donde sea necesario

La tabla va a quedar con estas columnas (en todas):
- categoria : Si es luz, gas, combustible o agua
- combustible: en caso de ser combustible si es 95,98, diesel o diesel premium
- precio: el coste en €
- unidad: la unidad del coste en € (si es por litro, metro cúbico o Megavatio hora)
- fecha: La fecha (en este caso el mes)

### Tabla Histórico_luz


In [39]:
df_luz.sample()

,index,precio,fecha
22,23,48.08,2020-11-01


In [40]:
df_luz = df_luz.drop(columns="index")
df_luz["categoria"] = "luz"
df_luz["combustible"] = "N/A"
df_luz["unidad"] = "€/MWh"
df_luz = df_luz[["categoria","combustible","precio","unidad","fecha"]]
df_luz.sample()

,categoria,combustible,precio,unidad,fecha
17,luz,N/A,35.98,€/MWh,2020-06-01


### Tabla Histórico_gas

In [41]:
df_gas.sample()

,index,precio_gas,fecha
18,19,7.74,2020-07-01


In [42]:
df_gas = df_gas.drop(columns="index")
df_gas["categoria"] = "gas"
df_gas["combustible"] = "N/A"
df_gas["unidad"] = "€/MWh"
df_gas = df_gas.rename(columns={"precio_gas":"precio"})
df_gas = df_gas[["categoria","combustible","precio","unidad","fecha"]]
df_gas.sample()

,categoria,combustible,precio,unidad,fecha
39,gas,N/A,113.77,€/MWh,2022-04-01


### Tabla Histórico_combustible

In [43]:
df_gasolina.sample()

,Fecha,Precio,combustible
98,2021-03-01,1.356433,Gasolina 95


In [44]:
df_gasolina["categoria"] = "combustible"
df_gasolina["unidad"] = "€/L"
df_gasolina = df_gasolina.rename(columns={"Fecha":"fecha",
                                          "Precio":"precio"})
df_gasolina = df_gasolina[["categoria","combustible","precio","unidad","fecha"]]
df_gasolina.sample()

,categoria,combustible,precio,unidad,fecha
178,combustible,Diésel,1.908723,€/L,2022-11-01


### Tabla Histórico_agua

In [45]:
df_agua.sample()
df_agua["categoria"] = "agua"
df_agua["combustible"] = "N/A"
df_agua["unidad"] = "€/m3"
df_agua = df_agua.rename(columns={"consumo (€/m3)": "precio"})
df_agua = df_agua[["categoria","combustible","precio","unidad","fecha"]]
df_agua.sample()

,categoria,combustible,precio,unidad,fecha
39,agua,N/A,1.8,€/m3,2022-04-01


# Unir las tablas en una

In [46]:
df_historico = pd.concat([df_luz,df_gas,df_gasolina,df_agua],ignore_index=True)

In [47]:
df_historico.sample()

,categoria,combustible,precio,unidad,fecha
6,luz,N/A,56.65,€/MWh,2019-07-01


In [48]:
df_historico.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336 entries, 0 to 335
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   categoria    336 non-null    object 
 1   combustible  336 non-null    object 
 2   precio       336 non-null    float64
 3   unidad       336 non-null    object 
 4   fecha        336 non-null    object 
dtypes: float64(1), object(4)
memory usage: 13.3+ KB


### Ordenar por fecha y añadirle un index (su id de la DB)

In [49]:
df_historico = df_historico.sort_values(by="fecha")
df_historico.reset_index(drop=True,inplace=True)
df_historico.index = df_historico.index + 1
df_historico.reset_index(inplace=True)
df_historico.head()

,index,categoria,combustible,precio,unidad,fecha
0,1,luz,N/A,67.910000,€/MWh,2019-01-01
1,2,combustible,Diésel Premium,1.259785,€/L,2019-01-01
2,3,combustible,Gasolina 95,1.228664,€/L,2019-01-01
3,4,agua,N/A,1.570000,€/m3,2019-01-01
4,5,gas,N/A,24.100000,€/MWh,2019-01-01


### Reemplazar categoria,combustible,unidad y fecha
Poniendo el id de su FK para la Base de Datos

### columna categoria

In [50]:
df_categorias

,index,categoria
0,1,luz
1,2,gas
2,3,combustible
3,4,agua


In [51]:
dictio_categoria = crear_dictio(df_categorias,"categoria")
df_historico = df_a_db(dictio_categoria,df_historico,"categoria")
df_historico.sample()

,index,categoria,combustible,precio,unidad,fecha
33,34,4,N/A,1.57,€/m3,2019-05-01


### columna combustible

In [52]:
df_combustibles

,index,combustible
0,1,N/A
1,2,Gasolina 95
2,3,Gasolina 98
3,4,Diésel
4,5,Diésel Premium


In [53]:
dictio_combustibles = crear_dictio(df_combustibles,"combustible")
df_historico = df_a_db(dictio_combustibles,df_historico,"combustible")
df_historico.sample()

,index,categoria,combustible,precio,unidad,fecha
176,177,3,2,1.303584,€/L,2021-02-01


### columna unidad

In [54]:
df_unidades

,index,unidad
0,1,€/L
1,2,€/m3
2,3,€/MWh


In [55]:
dictio_unidades = crear_dictio(df_unidades,"unidad")
df_historico = df_a_db(dictio_unidades,df_historico,"unidad")
df_historico.sample()

,index,categoria,combustible,precio,unidad,fecha
134,135,3,2,1.204331,1,2020-08-01


### columna fecha

In [56]:
df_fechas.sample()

,index,fecha
11,12,2019-12-01


In [57]:
dictio_fechas = crear_dictio(df_fechas,"fecha")
df_historico = df_a_db(dictio_fechas,df_historico,"fecha")
df_historico.sample()

,index,categoria,combustible,precio,unidad,fecha
154,155,4,1,1.65,2,23


# Guardamos y continuamos

In [59]:
df_historico.to_csv("../datos/05_TablasPreDB/05_tabla_historicos.csv")